In [ ]:
from ipywidgets import interact
import numpy as np

# %matplotlib inline
%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt


In [ ]:
# %run neoanalyzer.py {'1302'}
neuro.plot_tuning_curve(kind='evk_count')

In [ ]:
# LDA analysis
trode = 0
cond_offset = 18

plt.figure()
lda = LinearDiscriminantAnalysis(n_components=2)
X, y = neuro.make_design_matrix('evk_count', trode=trode)
X_r0 = X[y<8, :]
y_r0 = y[y<8]
X_r0 = lda.fit(X_r0, y_r0).transform(X_r0)
plt.subplot(1,2,1)
color=iter(cm.rainbow(np.linspace(0,1,len(np.unique(y_r0)))))
for k in range(len(np.unique(y_r0))):
   c = next(color)
   plt.plot(X_r0[y_r0==k, 0], X_r0[y_r0==k, 1], 'o', c=c, label=str(k))
plt.legend(loc='best')


X, y = neuro.make_design_matrix('evk_count', trode=trode)
trial_inds = np.logical_and(y>=9, y<17) # no control position
# trial_inds = np.logical_and(y>=18, y<26) # no control position
X_r0 = X[trial_inds, :]
y_r0 = y[trial_inds]
X_r0 = lda.fit(X_r0, y_r0).transform(X_r0)
color=iter(cm.rainbow(np.linspace(0,1,len(np.unique(y_r0)))))
plt.subplot(1,2,2)
for k in range(len(np.unique(y_r0))):
    c = next(color)
    plt.plot(X_r0[y_r0==k+9, 0], X_r0[y_r0==k+9, 1], 'o', c=c, label=str(k))
    plt.xlim(-6,6)
    plt.ylim(-8, 4)
#     plt.plot(X_r0[y_r0==k+9+9, 0], X_r0[y_r0==k+9+9, 1], 'o', c=c, label=str(k))
plt.legend(loc='best')
plt.show()

print(X_r0)

In [ ]:
# set-point analysis

def plot_setpoint(neuro, cond=0, color='k', error='sem'):
    
    ax = plt.gca()
    sp_temp = neuro.wt[cond][:, 1, :]
    mean_sp = np.mean(sp_temp, axis=1)
    se      = sp.stats.sem(sp_temp, axis=1)
    
    # inverse of the CDF is the percentile function. ppf is the percent point funciton of t.
    if error == 'ci':
        err = se*sp.stats.t.ppf((1+0.95)/2.0, sp_temp.shape[1]-1) # (1+1.95)/2 = 0.975
    elif error == 'sem':
        err = se

    plt.plot(neuro.wtt, mean_sp, color)
    plt.fill_between(neuro.wtt, mean_sp - err, mean_sp + err, facecolor=color, alpha=0.3)
    return ax

for k in range(neuro.control_pos):
    plt.figure()
    plot_setpoint(neuro, cond=k, color='k')
    plot_setpoint(neuro, cond=k+9, color='r')
    plot_setpoint(neuro, cond=k+9+9, color='b')
    plt.xlim(-0.5, 2.0)
    plt.xlabel('time (s)')
    plt.ylabel('set-point (deg)')
    plt.title('condition {}'.format(str(k)))
    plt.show()

# neuro.rates(kind='wsk_boolean')
# print(neuro.num_good_trials)

